# Imports

In [1]:
import jsonlines
import time

from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoTokenizer

/Users/danmueller/Documents/GitHub/aerospace_chatbot/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [ ]:
import torch
device=torch.device('mps')

# Read training data

In [ ]:
data_folder='../data/AMS/'
model='TheBloke/'
name='Llama-2-7B-Chat-GGUF'
file=data_folder+'ams_data_train_'+name+'-100.jsonl'

training_data = []
with jsonlines.open(file) as reader:
    for line in reader:
        training_data.append(line)

# Test cases

## Example straight from https://huggingface.co/docs/trl/v0.7.4/en/sft_trainer

In [ ]:
dataset = load_dataset("imdb", split="train")

trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
)
trainer.train()

## My data

In [ ]:
dataset=load_dataset('dsmueller/ams_data_train_Llama-2-7B-Chat-GGUF-LLM-generic_100',split='train')

In [ ]:
dataset[0]['text']

In [ ]:
model_name='TinyLlama/TinyLlama-1.1B-Chat-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokens=[]
for data in dataset:
    tokens.append(len(tokenizer.tokenize(data['text'])))
print(max(tokens))

In [ ]:
start_time=time.time()

trainer = SFTTrainer(
    model_name,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max(tokens),
)
trainer.train()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"The command took {elapsed_time:.4f} seconds to execute.")

Problem with not enough memory. Not likely to succeed here running locally on a mac on decent sized models.